### Choice of robustness metrics:
| Robustness metric 	 | value 	 |
|:-------------------:|:-------:|
|   test          	   | 0    	  |
|          	          |    	    |
|          	          |    	    |

In [2]:
#   import packages
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pickle
import functools

from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
    util,
    ScalarOutcome,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.em_framework.optimization import ArchiveLogger, EpsilonProgress
from ema_workbench.em_framework import parameters_from_csv


In [3]:
dike_model, planning_steps = get_model_for_problem_formulation(5)
ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

In [4]:
# List the names of vars to make `robustness_functions` a bit more read-able
var_list_damage = ['A.1_Expected Annual Damage',
                   'A.2_Expected Annual Damage',
                   'A.3_Expected Annual Damage',
                   'A.4_Expected Annual Damage',
                   'A.5_Expected Annual Damage',]
var_list_deaths = ['A.1_Expected Number of Deaths',
                   'A.2_Expected Number of Deaths',
                   'A.3_Expected Number of Deaths',
                   'A.4_Expected Number of Deaths',
                   'A.5_Expected Number of Deaths',]
var_list_dike = ['A.1_Dike Investment Costs',
                 'A.2_Dike Investment Costs',
                 'A.3_Dike Investment Costs',
                 'A.4_Dike Investment Costs',
                 'A.5_Dike Investment Costs',]
var_list_rfr = ['RfR Total Costs',]
var_list_evac = ['Expected Evacuation Costs',]

In [7]:
def sum_sum_sum(*data):
    sumsumsum = sum(sum(sum(data)))

    mean = np.mean(sumsumsum)
    iqr = sp.stats.iqr(sumsumsum) + mean * 0.005
    result = mean + iqr

    return result

# robustness metrics
maximize = ScalarOutcome.MAXIMIZE
minimize = ScalarOutcome.MINIMIZE

# These functions need to only return one score value...
robustness_functions = [
    ScalarOutcome('Damage Score', variable_name=var_list_damage,
                  function=sum_sum_sum, kind=minimize, expected_range=(0, 4e16)),
    ScalarOutcome('Deaths Score', variable_name=var_list_deaths,
                  function=sum_sum_sum, kind=minimize, expected_range=(0, 8.5e19)),
    ScalarOutcome('Dike Invest Score', variable_name=var_list_dike,
                  function=sum_sum_sum, kind=minimize, expected_range=(1e18, 1.3e7)),
    ScalarOutcome("RfR Invest Score", kind=minimize, variable_name="RfR Total Costs", function=sum_sum_sum),
    ScalarOutcome("Evac Score", kind=minimize, variable_name="Expected Evacuation Costs", function=sum_sum_sum),
]

constraints = [
    Constraint()
]

In [ ]:
from ema_workbench.em_framework.optimization import EpsilonProgress
from ema_workbench.em_framework.evaluators import BaseEvaluator

# general input
n_scenarios = 10
scenarios = sample_uncertainties(dike_model, n_scenarios)
nfe = 200 #make this large

BaseEvaluator.reporting_frequency = 0.1

epsilons = [0.05, ]*len(robustness_functions)

# Record the run time

# Run MORO
with MultiprocessingEvaluator(dike_model) as evaluator:
    results, convergence = evaluator.robust_optimize(robustness_functions,
                                                     scenarios=scenarios,
                                                     nfe=nfe,
                                                     epsilons=epsilons,
                                                     convergence=[
                                                         EpsilonProgress()],
                                                     population_size=5,
                                                     )

# Write the results so this step can be skipped when doing multiple analyzes
with open(r'generated_datasets/initial_Pareto_policies.pkl', 'wb') as file_pi:
    pickle.dump(results, file_pi)

[MainProcess/INFO] pool started with 12 workers
  2%|█                                         | 5/200 [00:12<08:17,  2.55s/it]

112790238.16708945 563951.1908354473
0.020874262862093176 0.00010437131431046589
6799603726.300705 33998018.63150352
6062000000.0 30310000.0
7756.5065218566015 38.78253260928301
14964097.452866822 74820.48726433411
0.0012325041313132755 6.162520656566377e-06
9411974141.233093 47059870.70616547
12996000000.0 64980000.0
1489.1048254493414 7.445524127246707
162512471.69085583 812562.3584542791
0.025148261108202077 0.0001257413055410104
7910344530.817875 39551722.65408938
7260000000.0 36300000.0
13057.89164696927 65.28945823484635
214022713.7406583 1070113.5687032915
0.19508086771298122 0.0009754043385649062
7879866945.239641 39399334.726198204
9701000000.0 48505000.0
0.0 0.0
176775294.9862554 883876.4749312771
0.02525271274772862 0.0001262635637386431
8905336493.451618 44526682.46725809
15174000000.0 75870000.0
14640.679566350716 73.20339783175358


In [ ]:
results

In [ ]:
# Extract the levers for each Pareto policy
policies = []
for row in range(results.shape[0]):
    policies.append(
        # Do not include the damage scores
        Policy(name=row, **results.iloc[row, :-5].to_dict())
    )

with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=policies)

# Write the results so this step can be skipped when doing multiple analyzes
with open('Outcomes/epsilon_results.pkl', 'wb') as file_pi:
    pickle.dump(results, file_pi)